In [1]:
from transformers import Wav2Vec2ForCTC

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import torch

In [3]:
import re

In [4]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।]'

In [6]:
model = Wav2Vec2ForCTC.from_pretrained('Saved_trainer_bengali_version-2')

In [7]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [8]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

In [10]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [11]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [12]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [13]:
from datasets import load_dataset

In [14]:
common_voice_test_transcription = load_dataset("/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn", split="test")

In [15]:
common_voice_test_transcription

Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'locale', 'segment'],
    num_rows: 8226
})

In [16]:
common_voice_test_transcription = common_voice_test_transcription.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [17]:
common_voice_test_transcription

Dataset({
    features: ['path', 'sentence'],
    num_rows: 8226
})

In [18]:
# common_voice_test_transcription = common_voice_test_transcription.train_test_split(train_size = 0.05)['train']

In [19]:
common_voice_test_transcription

Dataset({
    features: ['path', 'sentence'],
    num_rows: 8226
})

In [20]:
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, "", batch["sentence"]).lower()
    return batch

In [21]:
def prepare_dataset(batch):
    audio = batch['audio']
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch    

In [22]:
common_voice_test_transcription = common_voice_test_transcription.map(remove_special_characters)

In [23]:
common_voice_test_transcription[0]

{'path': 'common_voice_bn_31622386.mp3',
 'sentence': 'গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহ বন্দরটি দক্ষতার সাথে বিশ্বের বৃহত্তম বাল্ক ক্যারিয়ার পরিচালনা করতে সক্ষম'}

In [24]:
def add_path(record):
    home = '/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/'
    record['path'] = home+record['path']
    return record
    

In [25]:
common_voice_test_transcription = common_voice_test_transcription.map(add_path)

In [26]:
common_voice_test_transcription[0]

{'path': '/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31622386.mp3',
 'sentence': 'গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহ বন্দরটি দক্ষতার সাথে বিশ্বের বৃহত্তম বাল্ক ক্যারিয়ার পরিচালনা করতে সক্ষম'}

In [27]:
import os
import librosa

In [28]:
def add_audio(x):
    audio_data,sr = librosa.load(x['path'],sr=16000)
    return {'audio': {"array":audio_data,'sampling_rate':sr}}

In [29]:
common_voice_inference = common_voice_test_transcription.map(add_audio)

Map:   0%|          | 0/8226 [00:00<?, ? examples/s]

In [30]:
common_voice_inference.features

{'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)}}

In [32]:
import IPython.display as ipd

In [33]:
ipd.Audio(data=common_voice_inference[0]['audio']['array'], autoplay=True, rate=16000)


In [34]:
common_voice_inference[0]['sentence']

'গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহ বন্দরটি দক্ষতার সাথে বিশ্বের বৃহত্তম বাল্ক ক্যারিয়ার পরিচালনা করতে সক্ষম'

In [35]:
common_voice_inference = common_voice_inference.map(prepare_dataset)

Map:   0%|          | 0/8226 [00:00<?, ? examples/s]

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [36]:
# common_voice_inference['input_length'][0]

In [37]:
common_voice_inference.features

{'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)},
 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'input_length': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [38]:
def decoded_voice(encoded):
    input_dict = processor(encoded, return_tensors="pt", padding=True,sampling_rate=16000)
    logits = model(input_dict.input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    return processor.decode(pred_ids)

In [39]:
common_voice_inference['sentence'][15]

'এই গানের বিষয় ছিল সকাল বেলার পরিবেশ'

In [40]:
decoded_voice(common_voice_inference[15]["input_values"])

'এই গানের বিষয় ছিল সকাল বেলার পরিবেশ'

In [41]:
for i in range(5):
    print(f"Original value = {common_voice_inference[i]['sentence'].lower()}")
    print(f"Decoded value' = {decoded_voice(common_voice_inference[i]['input_values'])}")
    print('-------------------------')

Original value = গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহ বন্দরটি দক্ষতার সাথে বিশ্বের বৃহত্তম বাল্ক ক্যারিয়ার পরিচালনা করতে সক্ষম
Decoded value' = গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহবন্দরটি দক্ষতার সাথে বিশ্বের বৃহত্তম বাল ক্যারিয়ার পরিচালনা করতে সক্ষম
-------------------------
Original value = এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছেন তিনি
Decoded value' = এছাড়াও নীজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছেন তিনি
-------------------------
Original value = জন্য পরিকল্পিত
Decoded value' = জন্যব পরিকল্ভিত
-------------------------
Original value = বিভিন্ন আন্তর্জাতিক সংস্থার সদর দফতর এখানে অবস্থিত যাদের মধ্যে বিশ্ব পোস্টাল ইউনিয়নের নাম উল্লেখযোগ্য
Decoded value' = বিভিন্ন আন্তর্জাতিক সংস্থার সদরতফ্তর এখানে অবস্থিত যাদের মধ্যে বিশ্বপোষ্টাল ইউনিয়নের নাম উল্লেখযুক
-------------------------
Original value = যা তাদের কৃষিকাজ ব্যতীত অতিরিক্ত আয়ের উৎস
Decoded value' = যা তাদের কৃষিকাছ ব্যড়িত অতিরিক্ত আয়ের উঠস
-------------------------


In [42]:
import csv

In [43]:
with open("inference_bengali_v_2.csv", "w", newline="") as f:
    writer = csv.writer(f)

    
    writer.writerow(["Original value", "Decoded value"])

    for i in range(len(common_voice_inference)):
        original_value = common_voice_inference[i]["sentence"].lower()
        decoded_value = decoded_voice(common_voice_inference[i]["input_values"])

        writer.writerow([original_value, decoded_value])

        
f.close()

In [44]:
import pandas as pd

In [45]:
df = pd.read_csv('inference_bengali_v_2.csv')

In [46]:
df

,Original value,Decoded value
0,গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহ বন্দরটি...,গভীর জলের বার্থ ও বহুমুখী টার্মিনাল সহবন্দরটি ...
1,এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...,এছাড়াও নীজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছে...
2,জন্য পরিকল্পিত,জন্যব পরিকল্ভিত
3,বিভিন্ন আন্তর্জাতিক সংস্থার সদর দফতর এখানে অবস...,বিভিন্ন আন্তর্জাতিক সংস্থার সদরতফ্তর এখানে অবস...
4,যা তাদের কৃষিকাজ ব্যতীত অতিরিক্ত আয়ের উৎস,যা তাদের কৃষিকাছ ব্যড়িত অতিরিক্ত আয়ের উঠস
...,...,...
8221,অ্যাডমিরাল চ্যাটার্জী নতুন দিল্লিতে মারা যান,একগ্মিরাল চেটার জি নতুন দিল্লিতে মারা যান
8222,সিদ্ধান্ত নিলেন ব্যান্ড গড়ার,সিদ্ধান্ত নিলেন ব্যেন্ড গডার
8223,এই তারা সূর্য থেকে তৃতীয় নিকটবর্তী,এই তারা সূর্য থেকে তিতি অনিকর পরতি
8224,ঝরিয়ায় ভূমিকম্পের ফলে ভূগর্ভস্থ আগুন আরও ছড়...,জরি়ায় ভূমিকম্পের ফলে ভৌগোপ স্থাবু নআরছরি়ে পড়ে


In [47]:
from datasets import load_metric
wer = load_metric('wer')
cer = load_metric('cer')

/tmp/ipykernel_143409/1421994326.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer = load_metric('wer')


In [48]:
wer_score = wer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [49]:
wer_score

0.39131529430472

In [50]:
cer_score = cer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [51]:
cer_score

0.10334910568149322

In [52]:
from bnunicodenormalizer import Normalizer 
bnorm  = Normalizer()

In [53]:
def normalize(sen):
    sen = str(sen)
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

In [54]:
df['Decoded value']= df['Decoded value'].apply(lambda x:normalize(x))
df['Original value']= df['Original value'].apply(lambda x:normalize(x))

In [55]:
wer_score = wer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [56]:
wer_score

0.3873797471709114

In [57]:
cer_score = cer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [58]:
cer_score

0.10281820038747934